# Algorytmika i matematyka uczenia maszynowego 
## Laboratorium 9 - macierz odwrotna i wyznacznik macierzy


### Zadanie 1 - szyfr Hilla

Zaimplementuj dwie funkcje do szyfrowania i deszyfrowania wiadomości za pomocą szyfru [Hilla](https://pl.wikipedia.org/wiki/Szyfr_Hilla):
* `encrypt(t, K)` - szyfruje tekst $t$ za pomocą klucza (macierzy) $K$.
* `decrypt(s, K)` - deszyfruje kod $s$ za pomocą klucza (macierzy) $K$.

#### Szyfrowanie

W systemie kodowania ASCII litery A - Z zapisane są jako liczby z zakresu 65 - 90. Aby zaszyfrować tekst za pomocą klucza $K$ (macierz o wymiarach $m$ x $m$), należy zapisać znaki tekstu w postaci macierzy o wymiarach $m$ x $n$, a następnie wykonać następujące operacje:

1. **Utwórz macierz szyfrującą** $K$, której wyznacznik wynosi $det(K) = 1$.
> Uwaga: jest to odstępstwo od oryginalnego algorytmu mające na celu uproszczenie przykładu.
> Macierz taką można utworzyć z macierzy jednostkowej (`np.identity`), korzystając z operacji elementarnych np. dodając do jednego wiersza macierzy inny wiersz pomnożony przez skalar.
1. **Zamień litery tekstu $t$ o długości $h$ na wektor liczb**.
1. **Dopełnij wektor zerami** (_padding_), aby jego długość była podzielna przez $m$.
1. **Uformuj macierz $X$** o wymiarach ($m$ x $n$), gdzie $n = \lceil \frac{h}{m} \rceil$ (możesz użyć funkcji `reshape`).
1. **Wykonaj operację mnożenia macierzy**: $S = KX$.
1. **Spłaszcz macierz `S`** do wektora (np. za pomocą `flatten`) i zwróć szyfrogram `s` (zaszyfrowany tekst).


> Uwaga 1: Przedstawiony algorytm jest uproszczonym algorytmem, posiadającym ograniczenie $det(K)=1$, które można pominąć, ale wtedy należy do macierzy kodującej wyznaczyć macierz odwrotną modulo 26 (liczba znaków A-Z, ale może być dowolna inna). Podobnie, należy macierz $S$ zamienić na modulo 26. **Istotne**: W tym przypadku należy pamiętać, że wyznacznik macierzy szyfrującej $det(K)$ nie może posiadać wspólnego dzielnika z liczbą 26 (czyli obie liczby muszą być względnie pierwsze). Dlaczego? Bo w przeciwnym wypadku nie istnieje liczba odwrotna do $det(K) \textit{ mod } 26$.

> Uwaga 2: Szyfrogram $s$ może zawierać niedrukowalne znaki (gdy operujemy na kodach ASCII). Jeżeli chcesz tego uniknąć możesz np. zmapować znaki (65-90) do zakresu 0-25. Następnie przy wyświetlaniu przeprowadzić operację w drugą stronę.


#### Deszyfrowanie

Aby rozszyfrować zaszyfrowaną wiadomość $s$, wykonaj:

1. **Zamień ciąg $s$ na macierz $S$**.
1. **Oblicz macierz odwrotną** $K^{-1}$ (funkcja `np.linalg.inv`).
1. **Rozszyfruj wiadomość** wykonując operację $W = K^{-1} S$.
1. **Spłaszcz macierz** $W$ i zamień liczby z powrotem na znaki (ciąg tekstowy $w$).

Po poprawnym wykonaniu operacji powinieneś uzyskać $t_{1}t_{2}...t_{h} = w_{1}w_{2}...w_{h}$, gdzie $t$ to tekst oryginalny, a $w$ to tekst odszyfrowany, który może posiadać nadmiarowe symbole wynikające z dopełnienia (_padding_). Możesz je usunąć, ale nie jest to konieczne.


**Uwaga**: Jest to tylko laboratoryjny przykład na zastosowanie operacji macierzowych. Przedstawione rozwiązanie nie jest bezpieczne.

**Uwaga 2**: Dopilniuj, aby macierz szyfrująca nie przypominała macierzy jednostkowej!

In [1]:
import numpy as np
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

#Number to letter by division rest
def num_to_letter(num):
    return ALPHABET[num % 26]

#Letter to number by index
def letter_to_num(letter):
    letter = letter.upper()
    return ALPHABET.index(letter)

def text_to_vector(text):
    vector = []
    for e in text:
        vector.append(letter_to_num(e))
        
    return np.array(vector)

def vector_to_text(vector):
    text = []
    for e in vector:
        text.append(num_to_letter(e))
        
    return "".join(text)
        
###########################################################
def encrypt(message,key):
    m = key.shape[0]
    #2
    #Zamień litery tekstu o długości na wektor liczb.
    h = len(message)
    v = text_to_vector(message)
    
    #3
    #Dopełnij wektor zerami (padding), aby jego długość była podzielna przez m
    r = h % m
    if r !=0:
        for _ in range(r):
            v = np.append(v,0)
            
    #4
    #Uformuj macierz o wymiarach (m,n), gdzie (możesz użyć funkcji reshape)     
    h = len(v)
    n = int(h/m)
    x = v.reshape((m,n))
    #Wykonaj operację mnożenia macierzy: kx
    s = key @ x
    #Spłaszcz macierz S do wektora (np. za pomocą flatten) i zwróć szyfrogram s (zaszyfrowany tekst)
    s = s.flatten()
    return s
##########################################################################
def decrypt(message, key):
    #1
    #Zamień ciąg s na macierz S
    m = key.shape[0]
    h = len(message)
    
    n = int(h/m)
    S = message.reshape(m,n)
    #2
    #Oblicz macierz odwrotną (funkcja np.linalg.inv)
    k_inv = np.linalg.inv(key)
    #3
    #Rozszyfruj wiadomość
    W = k_inv @ S
    #4
    #Spłaszcz macierz i zamień liczby z powrotem na znaki (ciąg tekstowy )
    w = W.flatten().astype(int)
    w = vector_to_text(w)
    
    return w
##############################################################################


#Utwórz macierz szyfrującą K (uproszczone aby uniknac bledu modulo)

key = np.identity(2, dtype=int)
key[1] += 2 * key[0]

message = "ZakodowanaXXXWiadomosc"

encrypted = encrypt(message, key)
decryted = decrypt(encrypted, key)

print("Original message: ",message)
print("Encrypted - Decrypted message: ",decryted)


Original message:  ZakodowanaXXXWiadomosc
Encrypted - Decrypted message:  ZAKODOWANAXXXWIADOMOSC


### Zadanie 2

Zaimplementuj funkcję, która przyjmuje macierz kwadratową jako argument i zwraca jej wyznacznik obliczony zgodnie ze wzorem [Leibniza](https://en.wikipedia.org/wiki/Leibniz_formula_for_determinants) (_definicja permutacyjna_) i porównaj z wynikiem gotowej funkcji z biblioteki numpy `np.linalg.det`.

$$
\text{det}(A) = \sum_{\sigma \in S_n}\left(\text{sgn}(\sigma)\prod_{i=0}^{n-1}a_{i, \sigma(i)}\right)
$$

, gdzie:

* $S_n$ - [grupa permutacji](https://en.wikipedia.org/wiki/Symmetric_group) (dla macierzy 3x3 będą to permutacje ze zbioru {0, 1, 2})
* $\text{sgn}$ - jest to symbol "+", "-" w zależny od [parzystości permutacji](https://en.wikipedia.org/wiki/Parity_of_a_permutation). Np. dla permutacji `[1, 2, 0]` będzie to "+" (trzeba wykonać dwie operacje - zamienić `0` z `1` a później `1` z `2`, a dla permutacji `[0, 2, 1]` będzie "-" ponieważ wystarczy jedna operacja (zamiana `1` z `2`).
* $\sigma$ - permutacja (element z grupy permutacji $S_n$)

##### Przykład dla macierzy 3x3

| $\sigma$ | $\text{sgn}$ | $\text{sgn}(\sigma)\prod_{i=0}^{n-1}a_{i, \sigma(i)}$ |
| :---     | :---         | ---: |
| 1, 2, 3  | +            | $+a_{1,1}a_{2,2}a_{3,3}$ |
| 1, 3, 2  | -            | $-a_{1,1}a_{2,3}a_{3,2}$ |
| 3, 1, 2  | +            | $+a_{1,3}a_{2,1}a_{3,2}$ |
| 3, 2, 1  | -            | $-a_{1,3}a_{2,2}a_{3,1}$ |
| 2, 3, 1  | +            | $+a_{1,2}a_{2,3}a_{3,1}$ |
| 2, 1, 3  | -            | $-a_{1,2}a_{2,1}a_{3,3}$ |

$\text{det}(A) = a_{1,1}a_{2,2}a_{3,3} - a_{1,1}a_{2,3}a_{3,2} + a_{1,3}a_{2,1}a_{3,2} - a_{1,3}a_{2,2}a_{3,1} + a_{1,2}a_{2,3}a_{3,1} - a_{1,2}a_{2,1}a_{3,3}$

> Uwaga 1: Aby sprawdzić parzystość permutacji możesz użyć funkcji `parity` z biblioteki [`sympy`](https://docs.sympy.org/latest/modules/combinatorics/permutations.html#sympy.combinatorics.permutations.Permutation.parity) (aby zainstalować uruchom `pip install sympy`). Przykład: 
```python
from sympy.combinatorics import Permutation
Permutation([0, 2, 1]).parity()
```
    
> Uwaga 2: W celu wygenerowania permutacji możesz użyć funkcji `permutations` z modułu [`itertools`](https://docs.python.org/3/library/itertools.html#itertools.permutations)

> Uwaga 3: Pamiętaj, że w numpy porównywanie liczb zmiennoprzecinkowych wykonuje się za pomocą funkcji [`allclose`](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html)

In [2]:
import itertools
#################3
#Czesc z GPT (Nie chcialem dodawac kolejnej biblioteki)
def permutation_sign(p):
    return 1 if sum(p[i] > p[j] for i in range(len(p)) for j in range(i+1, len(p))) % 2 == 0 else -1
#########################3

def Own_Det_Leibniz(Matrix):
    shape = Matrix.shape[0]
    
    x = np.arange(0,shape,1).tolist()
    permutations = list(itertools.permutations(x))
    
    
    
    det = 0
    for p in permutations:
        det_update = []
        for i,e in enumerate(p):
            det_update.append(Matrix[i,e])
            
        #Mnozenie    
        sign = permutation_sign(p)
        det_update = np.prod(det_update)*sign
    
        det = det+ det_update
        
    return det

############
M = np.array([[1,2,4],
             [1,0,2],
             [3,2,1]])
#numpy vs own
w_np = np.linalg.det(M)
w_own = Own_Det_Leibniz(M)
print("Wyznacznik macierzy numpy:", round(w_np,3))
print("Wyznacznik macierzy wlasna funkcja:", round(w_own,3))

Wyznacznik macierzy numpy: 14.0
Wyznacznik macierzy wlasna funkcja: 14
